# Create RNN Model


![](img/gru_rnn_model.png)

Considerando o batch_size ainda indeterminado (`None`)  a entrada na rede é um tensor de índices de palavras de dimensão `(None, MAX_SEQLEN, 1)`. Ele é processado através da camada **Embedding**, que converte cada palavra num vetor da forma `(EMBED_SIZE)`, pelo qual o tensor de saída desta camada tem a dimensão `(None, MAX_SEQLEN, EMBED_SIZE)`. O tensor é a entrada ao encoder GRU e sai com uma dimensão `HIDDEN_SIZE`. A camada GRU é configurada para retornar um único vetor de contexto (`return_sequences=False`) depois de observar uma seqüência de dimensão `MAX_SEQLEN`, então o tensor de saída da camada GRU tem a forma `(None, HIDDEN_SIZE)`. 

O vetor de contexto é logo replicado utilizando a camada `RepeatVector` em um tensor da forma `(None, MAX_SEQLEN, HIDDEN_SIZE)` e é a entrada ao decoder GRU. Logo passa por uma camada FC que produze um vetor de saída de dimensão `(None, MAX_SEQLEN,t_vocab_size)`. A função de ativação na camada FC é `softmax`. O argumento máximo `(argmax)` em cada coluna do tensor é indexada para prever o POS tag para a palavra em essa posição.

A definição do modelo é feita por : `EMBED_SIZE, HIDDEN_SIZE, BATCH_SIZE` e `NUM_EPOCHS` são hiperparametros que tem sido atribuído com estes valores depois de experimentar com diferentes valores. O modelo é compilado com a função de perda `categorical_crossentropy` dado que temos multiplex categorias de rótulos, e o otimizador é utilizado o popular otimizador `adam`.

In [4]:
%run '3.a. POS_tagging.ipynb'

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla C2070 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN not available)


10947 249 3914 45 249 3914


In [5]:
from keras.layers.core import Activation, Dense, Dropout, RepeatVector, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import TimeDistributed
from keras.models import Sequential

In [6]:
EMBED_SIZE = 128
HIDDEN_SIZE = 64
BATCH_SIZE = 32
NUM_EPOCHS = 1

## Tarefa: Comparar GRU, LSTM e SimpleRNN

In [57]:
from keras.layers.recurrent import GRU
np.random.seed(43)

In [58]:
model = Sequential()
model.add(Embedding(s_vocabsize, EMBED_SIZE, input_length=MAX_SEQLEN, embeddings_initializer="glorot_uniform"))
#model.add(SpatialDropout1D(0.2))
model.add(GRU(HIDDEN_SIZE, dropout=0.2, recurrent_dropout=0.2))
model.add(RepeatVector(MAX_SEQLEN))
model.add(GRU(HIDDEN_SIZE, return_sequences=True))
model.add(TimeDistributed(Dense(t_vocabsize)))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Treinamos este modelo para uma única época. O modelo tem muito parâmetros, e começa a ter overfit apos a primeira época de treinamento. Quando fornecemos a mesma data várias vezes ao longo das épocas, o modelo começa a ter um overfit nos dados de treinamento e piora os resultados com os dados de validação.

In [59]:
model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_data=[Xtest, Ytest], 
          verbose=0, callbacks=[TQDMNotebookCallback()])
score, acc = model.evaluate(Xtest, Ytest, batch_size=BATCH_SIZE, verbose=0)
print("Test score: %.4f, accuracy %.4f" % (score, acc))


Test score: 0.2929, accuracy 0.9109


Os models **Sequence-to-Sequence** são uma classe de modelos muito poderosos. Seu maior aplicação são maquinas de tradução, mais existe muitas outras aplicações como os exemplos anteriores. Muitas das tarefas de NLP, são maiores na hierarquia como:

* **Named Entity Recognition** (para maior informação ler: [Named Entity Recofnition with Long Short Term Memory](http://delivery.acm.org/10.1145/1120000/1119202/p172-hammerton.pdf?ip=139.82.47.87&id=1119202&acc=OPEN&key=344E943C9DC262BB%2E5C4D48D3482FC7F0%2E4D4702B0C3E38B35%2E6D218144511F3437&CFID=767933077&CFTOKEN=20089988&__acm__=1495992801_1ee6b5e50f105ad8a61b0a59dd162da6), Hammerton, Proceeding of the Seventh Conference on Natural Language Learning, 2003).
* **Sentence Parsing** (para maior informação ler os artigos: [Grammar as Foreing Language](https://arxiv.org/abs/1412.7449), O. Vinyals, Advances in Neural Information Processing Systems, 2015). 
* **Image captioning**: Também em redes de maior complexidade (para maior informação ler: [Deep Visual-Semantic Alignments for Generating Image Descriptions](https://cs.stanford.edu/people/karpathy/cvpr2015.pdf), A. Karpathy, e F. Li, Proceedings of the IEEE Conference on COmputer VIsion and Pattern Recognition, 2015).

# Bidirectional RNN

Em um determinado tempo *t*, a saida da RNN é dependente a todos os passos de tempo anteriores. Embora, é completamente possível que a saída seja também dependente de uma futura saída. Isto acontece especialmente em aplicações de NLP, onde talvez os atributos de uma palavra ou frases que tentamos predecir dependam do contexto dada pela frase completa, não só das palavras que precediam. RNNs bidirecionais também ajudam a arquitetura da rede dar igual importância ao início e final de a seqüência, e incrementa os dados disponíveis para treinamento.

RNN bidirecionais são 2 RNN acoplados uma acima de outra, lendo a entrada no sentido oposto. Asim, em nosso exemplo, uma RNN vai ler as palavras de esquerda a direita e a outra RNN vai ler as palavras de direita a esquerda. A saída em cada intervalo de tempo sera baseado nos estados das camadas ocultas de ambas RNN.

In [60]:
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
np.random.seed(43)

In [61]:
EMBED_SIZE = 128
HIDDEN_SIZE = 64
BATCH_SIZE = 32
NUM_EPOCHS = 1

model = Sequential()
model.add(Embedding(s_vocabsize, EMBED_SIZE, input_length=MAX_SEQLEN, embeddings_initializer="glorot_uniform"))
#model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, dropout=0.2, recurrent_dropout=0.2)))
model.add(RepeatVector(MAX_SEQLEN))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(t_vocabsize)))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [62]:
model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_data=[Xtest, Ytest], 
          verbose=0, callbacks=[TQDMNotebookCallback()])
score, acc = model.evaluate(Xtest, Ytest, batch_size=BATCH_SIZE, verbose=0)
print("Test score: %.4f, accuracy %.4f" % (score, acc))


Test score: 0.2797, accuracy 0.9116
